In [ ]:
from utils import make_roc_curve, make_cm, get_stats
import pandas as pd
from pathlib import Path
predictions = Path('MR_model_predictions_anonymized.csv')
manifest = pd.read_csv(predictions)

In [2]:
make_roc_curve(manifest)
make_cm(manifest)
get_stats(manifest)

KeyboardInterrupt: 